In [133]:
import pandas as pd


In [134]:
df = pd.read_csv(r'/content/drive/My Drive/jena_climate_2009_2016.csv')

In [135]:
df.head()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
0,01.01.2009 00:10:00,996.52,-8.02,265.40,-8.90,93.3,3.33,3.11,0.22,1.94,3.12,1307.75,1.03,1.75,152.3
1,01.01.2009 00:20:00,996.57,-8.41,265.01,-9.28,93.4,3.23,3.02,0.21,1.89,3.03,1309.80,0.72,1.50,136.1
2,01.01.2009 00:30:00,996.53,-8.51,264.91,-9.31,93.9,3.21,3.01,0.20,1.88,3.02,1310.24,0.19,0.63,171.6
3,01.01.2009 00:40:00,996.51,-8.31,265.12,-9.07,94.2,3.26,3.07,0.19,1.92,3.08,1309.19,0.34,0.50,198.0
4,01.01.2009 00:50:00,996.51,-8.27,265.15,-9.04,94.1,3.27,3.08,0.19,1.92,3.09,1309.00,0.32,0.63,214.3


In [136]:
df.tail()

,Date Time,p (mbar),T (degC),Tpot (K),Tdew (degC),rh (%),VPmax (mbar),VPact (mbar),VPdef (mbar),sh (g/kg),H2OC (mmol/mol),rho (g/m**3),wv (m/s),max. wv (m/s),wd (deg)
420546,31.12.2016 23:20:00,1000.07,-4.05,269.10,-8.13,73.10,4.52,3.30,1.22,2.06,3.30,1292.98,0.67,1.52,240.0
420547,31.12.2016 23:30:00,999.93,-3.35,269.81,-8.06,69.71,4.77,3.32,1.44,2.07,3.32,1289.44,1.14,1.92,234.3
420548,31.12.2016 23:40:00,999.82,-3.16,270.01,-8.21,67.91,4.84,3.28,1.55,2.05,3.28,1288.39,1.08,2.00,215.2
420549,31.12.2016 23:50:00,999.81,-4.23,268.94,-8.53,71.80,4.46,3.20,1.26,1.99,3.20,1293.56,1.49,2.16,225.8
420550,01.01.2017 00:00:00,999.82,-4.82,268.36,-8.42,75.70,4.27,3.23,1.04,2.01,3.23,1296.38,1.23,1.96,184.9


In [137]:
f = open(r'/content/drive/My Drive/jena_climate_2009_2016.csv')
data = f.read()
f.close()

In [138]:
data = data.split('\n')

In [139]:
header = data[0].split(',')
lines = data[1:]


In [140]:
import numpy as np
float_data = np.zeros((len(lines),len(header)-1))

In [141]:
for i,line in enumerate(lines):
  values = [float(x) for x in line.split(',')[1:]]
  float_data[i,:] = values

In [142]:
# mean = float_data.mean(axis=0)
# float_data-=mean
# std = float_data.std(axis=0)
# float_data /=std
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
float_data=scaler.fit_transform(float_data)

In [143]:
# float_data

In [144]:
training_data = float_data[0:417600,:]
test_data = float_data[417600:420480,:]

In [145]:
import numpy as np
test_data.shape

(2880, 14)

In [146]:
training_data = training_data.reshape(580,720,14)

In [147]:
test_data =test_data.reshape(4,720,14)

In [148]:
def create_dataset(dataset,timesteps=1):
  Y=[]
  X = np.zeros((dataset.shape[0]-1,dataset.shape[1],dataset.shape[2]))
  for i in range(len(dataset)-1):
    a=dataset[i,i:(i+timesteps),:]
    X[i,i:(i+timesteps),:] = a
    Y.append(dataset[i+1,i,13])
  return X,np.array(Y)

In [149]:
X_train,Y_train = create_dataset(training_data,720)

In [150]:
X_train.shape

(579, 720, 14)

In [151]:
Y_train.shape

(579,)

In [152]:
X_test,Y_test = create_dataset(test_data,720)

In [154]:

import keras
callbacks_list = [keras.callbacks.ModelCheckpoint(
filepath='my_model.h5',
monitor='val_loss',
save_best_only=True),
keras.callbacks.EarlyStopping(
    monitor='val_loss',patience=1
)
]

In [155]:

from keras.layers import LSTM,Flatten,Dense
from keras import Input
from keras.models import Model

In [156]:
input = Input(shape=(720,14))
Layer_1 = LSTM(50,activation='relu',return_sequences=True)(input)
Layer_2 = LSTM(50,activation='relu',return_sequences=True)(Layer_1)
Layer_3 = LSTM(50,activation='relu')(Layer_2)
Layer_4 = Flatten()(Layer_3 )
Layer_5 = Dense(12,activation='relu')(Layer_4 )
output = Dense(1)(Layer_5)
model = Model(input,output)

In [157]:

model.compile(loss='mean_squared_error',optimizer='Adam')

In [158]:
model.fit(X_train,Y_train,epochs=20,validation_split=0.2,callbacks=callbacks_list)

Epoch 1/20
15/15 [==============================] - 16s 1s/step - loss: 0.1557 - val_loss: 0.0510
Epoch 2/20
15/15 [==============================] - 15s 1s/step - loss: 0.0636 - val_loss: 0.0513


In [159]:
test_predict = model.predict(X_test)

In [160]:
test_predict

array([[0.44850725],
       [0.44763345],
       [0.46910042]], dtype=float32)

In [161]:
Y_test

array([0.51611111, 0.44972222, 0.61722222])